## Sentiment Analysis on Feedback Data

### Here we will apply some bag-of-words techniqes to caculate sentiment of the feedback text. We are going to try out two different algoriths

* nltk-vader
* textblob

### Data Loading & Preparation

Here we will be using individual text examples to test our code. If required, you can test ingest data from some file or database to test the code

### What question we are trying to answer?

1. Which one works better, nltk or textblob?

2. What is the advantage with bag of words approach?

3. When is bag of words not a good option?

### Analysis

Loading the get_sentiment function from sentiment module.
For more details on get_sentiment function, feel free to check out sentiment.py

In [1]:
from sentiment import get_sentiment

In [2]:
text =   "I am happy with the services provided"

Trying the "nltk" algo 

In [3]:
get_sentiment(text, algo="nltk")

0.5719

Trying the "textblob" algo

In [4]:
get_sentiment(text, algo="textblob")

0.8

Trying on some more examples, feel free to add your own

In [5]:
text =  "I am frustrated at not being able to reach your executive"


In [6]:
get_sentiment(text, algo="nltk")

-0.5106

In [7]:
get_sentiment(text, algo="textblob")

-0.09999999999999998

In [8]:
text = "I have received the required file"

In [9]:
get_sentiment(text, algo="nltk")

0.0

In [10]:
get_sentiment(text, algo="textblob")

0.0

### Answers

1. Which one works better, nltk or textblob?

Although we tested with very limited sentences, but nltk seems to be working better than textblob. The difference is negligible with positive feedbacks but nltk seems to be producing better results with negative feedback data. You can also test with sentences from different domains and compare the results.

2. What is the advantage with bag of words approach?

Sentiment Analysis suffers from lack of availability of labelled dataset. Labelling data is a tiresome and expensive process and such datasets may not be available for many domains. Bag of words is a very simple approach which can be used for sentiment analysis when getting labelled dataset is not a viable option.

3. When is bag of words not a good option?

Bag of words may not be a good option for certain domains. For example, let us test few examples from financial domain:


In [11]:
text = "I want to know the current price of the share"
get_sentiment(text, algo="nltk")

0.3612

In [12]:
text = "We want to discuss about the new bond issue"
get_sentiment(text, algo="textblob")

0.13636363636363635

So, we notice that although both the sentences are neutral in financial sense but out algorithms, both nltk and vader, think they carry some sentiment. Hence, bag-of-words may not work from some domains like Finance, Medical etc.